In [246]:

import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt

# import io
# import msoffcrypto

from pathlib import Path
#from tqdm import tqdm

from IPython.display import display_markdown
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None) 


In [247]:
oblako = pd.read_excel('Файлы_для_облака/23-10-2024_16-00_Реестр ЦПП_итог.xlsx')

In [248]:
# %store -r oblako oblako_name oblako_path oblako_spravochniki

df_obl = oblako.copy()
df_obl.shape

(18878, 51)

In [249]:
# ----------------------------- нормализуем даты 
def date_norm(df, date_coloumns_list):
    for i_ in date_coloumns_list:
        df[i_] = pd.to_datetime(df_obl[i_], dayfirst=True, errors = 'coerce').dt.normalize()
    return df

In [250]:
# ----------------------------- нормализуем даты 
date_list = [
'Дата включения пациента в Реестр', 
'Плановая дата первичного визита 24 часа', 
'Плановая дата первичного визита 72 часа', 
'Плановая дата госпитализации',
'Дата фактической госпитализации из  ЕМИАС', 
'Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС', 
'Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС', 
    
'Дата смерти'
, 'Последний визит врача из протокола осмотра ЕМИАС'    
    , 'Плановая дата следующего визита из протокола осмотра ЕМИАС'
    , 'Дата прекращения оказания ПМП'
]
    

In [251]:
# ----------------------------- нормализуем даты 
df_obl = date_norm(df_obl, date_list)

In [252]:
# --------------------------------------- удалим пробелы -------
df_obl.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [253]:
# df_obl.to_excel('df_obl_17окт.xlsx')

<div class="alert alert-block alert-warning">    <font size=3>
<b>БЛОКИ 1...

####
<div class="alert alert-block alert-info">    <font size=3>
<b>1.1. Просрочка 24 ч

In [254]:
# ------------------------------------------------------------------------------ НАШИ ПАЦИЕНТЫ -----------------------------
uslovie_1_наши =    ( df_obl['Задание по пациенту'].isin(['отделение выездной паллиативной службы']) 
                & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"')
                & (df_obl['Дата включения пациента в Реестр'] > '2024-06-01') 
               )

<div class="alert alert-block alert-info">    <font size=2>
    1 - просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
  

In [255]:
#  -------------------------------------------------------------------- 1.1. Просрочка 48 ч
var_48_1 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
         & df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
         & ((pd.to_datetime("today") - df_obl['Плановая дата первичного визита 24 часа']) > '2 days'))

var_48_2 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
         & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
         & ((df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] - df_obl['Плановая дата первичного визита 24 часа']) > '2 days'))

var_48_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении', 'Сопровождение приостановлено'])  
var_48_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  


In [256]:
# -------------------------------------------------------------------------1.1. Просрочка 48 ч
df_obl[ uslovie_1_наши 
       & (
             (var_48_1 | var_48_2 )
           & var_48_3 
           & var_48_4 
         )  
      ].shape

(1, 51)

In [257]:
df_obl.loc[ uslovie_1_наши 
       & (
             (var_48_1 | var_48_2 )
           & var_48_3 
           & var_48_4 
         )  
      , 'Правки' ] = '01 -просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    2 - ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
  

In [258]:
# #  -------------------------------------------------------------------ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
# var_24_1 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
#          & df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
#          & ((pd.to_datetime("today") - df_obl['Плановая дата первичного визита 24 часа']) > '1 days'))

# var_24_2 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
#          & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
#          & ((df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] - df_obl['Плановая дата первичного визита 24 часа']) > '1 days'))

# var_24_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении', 'Сопровождение приостановлено'])  
# var_24_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  


In [259]:
#  -------------------------------------------------------------------ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
var_24_1 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
         & df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
         & ((pd.to_datetime("today") - df_obl['Плановая дата первичного визита 24 часа']) == '2 days'))

var_24_2 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
         & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
         & ((df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] - df_obl['Плановая дата первичного визита 24 часа']) == '2 days'))

var_24_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении', 'Сопровождение приостановлено'])  
var_24_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  


In [260]:
# --------------------------------------------------------------------ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
df_obl[ uslovie_1_наши 
       & (
                (var_24_1 | var_24_2 )
               & var_24_3 
               & var_24_4 
         )  
      ].shape

(1, 52)

In [261]:
# ------------------------------------------------------------------ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении
df_obl.loc[ uslovie_1_наши 
       & (
                (var_24_1 | var_24_2 )
               & var_24_3 
               & var_24_4 
         )  
            , 'Правки' ] = '02 -ЗАВТРА просрочка по первичке 24 ч и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    3 - фактическая дата первички 24 ч меньше плановой даты первички 24 ч
  

In [262]:
#  ---------------------------- фактическая дата первички 24 ч меньше плановой даты первички 24 ч
var_факт_до_плана_24 = ( ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
                         & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
                         & (df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] < df_obl['Плановая дата первичного визита 24 часа']))

In [263]:
 # ---------------------------- факт_до_плана_24
df_obl[   uslovie_1_наши 
       &    
          var_факт_до_плана_24].shape

(4, 52)

In [264]:
 # ---------------------------- факт_до_плана_24
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_до_плана_24 , 'Правки' ] = '03 -фактическая дата первички 24 ч меньше плановой даты первички 24 ч'

<div class="alert alert-block alert-info">    <font size=2>
  4 - фактическая дата первички 24 ч из будущего

In [265]:
# -------------- фактическая дата первички 24 ч из будущего
var_факт_из_будущ_24 = (   ~df_obl['Плановая дата первичного визита 24 часа'].isna() 
                         & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
                         & (df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'] > pd.to_datetime("today"))
                        )

In [266]:
# -------------- фактическая дата первички 24 ч из будущего
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_из_будущ_24].shape

(0, 52)

In [267]:
# -------------- фактическая дата первички 24 ч из будущего
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_из_будущ_24, 'Правки' ] = '04 -фактическая дата первички 24 ч из будущего'

####
<div class="alert alert-block alert-info">    <font size=3>
<b>1.2. Просрочка 72 ч

<div class="alert alert-block alert-info">    <font size=2>
    5 - просрочка по первичке 72 ч и нет причины и/или статус новый/на сопровождении

In [268]:
#  -------------------------------------------------------------------- 1.2. Просрочка 72 ч ------------ визит 3 суток
var_72_1 = ( ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
         & df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
         & ((pd.to_datetime("today") - df_obl['Плановая дата первичного визита 72 часа']) > '3 days'))

var_72_2 = ( ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
         & ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
         & ((df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'] - df_obl['Плановая дата первичного визита 72 часа']) > '3 days'))

var_72_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении', 'Сопровождение приостановлено'])  
var_72_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  

In [269]:
# ------------------------------------------------------------------------ 1.2. Просрочка 72 ч ---------- визит 3 суток
df_obl.loc[ uslovie_1_наши 
       & (
                (var_72_1 | var_72_2 )
               & var_72_3 
               & var_72_4 
         )  
      ].shape

(146, 52)

In [270]:
# ------------------------------------------------------------------------ 1.2. Просрочка 72 ч ---------- визит 3 суток
df_obl.loc[ uslovie_1_наши 
       & (
                (var_72_1 | var_72_2 )
               & var_72_3 
               & var_72_4 
         )  
      , 'Правки' ] = '05 -просрочка по первичке 72 ч и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    6 - ЗАВТРА просрочка по первичке 72 ч и нет причины и/или статус новый/на сопровождении

In [271]:
#  -------------------------------------------------------------------- 1.2. Просрочка 72 ч ------------ визит 4 суток
var_72х4_1 = ( ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
         & df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
         & ((pd.to_datetime("today") - df_obl['Плановая дата первичного визита 72 часа']) == '2 days'))

var_72х4_2 = ( ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
                 & ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
                 & ((df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'] - df_obl['Плановая дата первичного визита 72 часа']) == '2 days')
             )

var_72х4_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении', 'Сопровождение приостановлено'])    
var_72х4_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  

In [272]:
# ------------------------------------------------------------------------ 1.2. Просрочка 72 ч ---------- визит 4 суток
df_obl.loc[ uslovie_1_наши 
       & (
                (var_72х4_1 | var_72х4_2 )
               & var_72х4_3 
               & var_72х4_4 
         )  
      ].shape

(35, 52)

In [273]:
# ------------------------------------------------------------------------ 1.2. Просрочка 72 ч ---------- визит 4 суток
df_obl.loc[ uslovie_1_наши 
       & (
                (var_72х4_1 | var_72х4_2 )
               & var_72х4_3 
               & var_72х4_4 
         )  
      , 'Правки' ] = '06 -ЗАВТРА просрочка по первичке 72 ч и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    7 - фактическая дата первички 24 ч меньше плановой даты первички 24 ч

In [274]:
# ---------------------------- факт_до_плана_72 ---- 72
var_факт_до_плана_72 = (   ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
                         & ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
                         & (df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'] < df_obl['Плановая дата первичного визита 72 часа'])
                       )

In [275]:
 # ---------------------------- факт_до_плана_72
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_до_плана_72].shape

(1703, 52)

In [276]:
 # ---------------------------- факт_до_плана_72
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_до_плана_72 , 'Правки' ] = '07 -фактическая дата первички 24 ч меньше плановой даты первички 24 ч'

<div class="alert alert-block alert-info">    <font size=2>
    8 - фактическая дата первички 72 ч из будущего

In [277]:
# -------------- фактическая дата первички 72 ч из будущего
var_факт_из_будущ_72 = (   ~df_obl['Плановая дата первичного визита 72 часа'].isna() 
                         & ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
                         & (df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'] > pd.to_datetime("today"))
                        )

In [278]:
# -------------- фактическая дата первички 72 ч из будущего
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_из_будущ_72].shape

(0, 52)

In [279]:
# -------------- фактическая дата первички 72 ч из будущего
df_obl.loc[   uslovie_1_наши 
       &    
          var_факт_из_будущ_72, 'Правки' ] = '08 -фактическая дата первички 72 ч из будущего'

####
<div class="alert alert-block alert-info">    <font size=3>
<b>1.3. Просрочка ГОСПИТ

In [280]:
# ------------------------------------------------------------------------------ НАШИ ПАЦИЕНТЫ ГОСПИТ -----------------------------
uslovie_наши_госпит =    ( df_obl['Задание по пациенту'].isin(['стационар паллиативное отделение',
                                                             'Стационар ЦПП',
                                                             'стационар ЦПП',
                                                             'Стационар паллиативное отделение',]) 
                # & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"')
                & (df_obl['Дата включения пациента в Реестр'] > '2024-06-01') 
                         )

<div class="alert alert-block alert-info">    <font size=2>
    9 - просрочка по первичке госп и нет причины и/или статус новый/на сопровождении

In [281]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 4 суток
var_госпит_1 = ( ~df_obl['Плановая дата госпитализации'].isna() 
                 & df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                 & ((pd.to_datetime("today") - df_obl['Плановая дата госпитализации']) > '4 days'))

var_госпит_2 = ( ~df_obl['Плановая дата госпитализации'].isna() 
                 & ~df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                 & ((df_obl['Дата фактической госпитализации из  ЕМИАС'] - df_obl['Плановая дата госпитализации']) > '4 days')
             )

var_госпит_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении'])  
var_госпит_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  

In [282]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 4 суток
df_obl.loc[
        uslovie_наши_госпит
       & (
                (var_госпит_1 | var_госпит_2 )
               & var_госпит_3 
               & var_госпит_4 
         )  
      ].shape

(74, 52)

In [283]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 4 суток
df_obl.loc[
        uslovie_наши_госпит
       & (
                (var_госпит_1 | var_госпит_2 )
               & var_госпит_3 
               & var_госпит_4 
         )  
      , 'Правки' ] = '09 -просрочка по первичке госп и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    10 - ЗАВТРА просрочка по первичке госп и нет причины и/или статус новый/на сопровождении

In [284]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 3 суток
var_госпит_3сут_1 = ( ~df_obl['Плановая дата госпитализации'].isna() 
                         & df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                         & ((pd.to_datetime("today") - df_obl['Плановая дата госпитализации']) > '3 days'))

var_госпит_3сут_2 = ( ~df_obl['Плановая дата госпитализации'].isna() 
                         & ~df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                         & ((df_obl['Дата фактической госпитализации из  ЕМИАС'] - df_obl['Плановая дата госпитализации']) > '3 days')
                     )

var_госпит_3сут_3 =  df_obl['Текущий статус пациента'].isin(['Новый (если назначен первичный визит)', 'На сопровождении',  'на сопровождении'])  
var_госпит_3сут_4 =  df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()  

In [285]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 3 суток
df_obl.loc[
        uslovie_наши_госпит
       & (
                (var_госпит_3сут_1 | var_госпит_3сут_2 )
               & var_госпит_3сут_3 
               & var_госпит_3сут_4 
         )  
      ].shape

(79, 52)

In [286]:
#  -------------------------------------------------------------------- 1.3. Просрочка госпитализации ------------ госпит 3 суток
df_obl.loc[
        uslovie_наши_госпит
       & (
                (var_госпит_3сут_1 | var_госпит_3сут_2 )
               & var_госпит_3сут_3 
               & var_госпит_3сут_4 
         )  
      , 'Правки' ] = '10 -ЗАВТРА просрочка по первичке госп и нет причины и/или статус новый/на сопровождении'

<div class="alert alert-block alert-info">    <font size=2>
    11 - фактическая дата первичной госп меньше плановой даты первичной госп

In [287]:
# ---------------------------- факт_до_плана_ГОСПИТ
var_факт_до_плана_госпит = (      ~df_obl['Плановая дата госпитализации'].isna() 
                                 & ~df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                                 & (df_obl['Дата фактической госпитализации из  ЕМИАС'] < df_obl['Плановая дата госпитализации'])
                           )

In [288]:
 # ---------------------------- факт_до_плана_ГОСПИТ
df_obl.loc[   uslovie_наши_госпит 
       &    
          var_факт_до_плана_госпит].shape

(129, 52)

In [289]:
 # ---------------------------- факт_до_плана_ГОСПИТ
df_obl.loc[   uslovie_наши_госпит 
       &    
          var_факт_до_плана_госпит, 'Правки' ] = '11 -фактическая дата первичной госп меньше плановой даты первичной госп'

<div class="alert alert-block alert-info">    <font size=2>
    12- фактическая дата первички _ГОСПИТ из будущего

In [290]:
# -------------- фактическая дата первички _ГОСПИТ из будущего
var_факт_из_будущ_госпит =  (      ~df_obl['Плановая дата госпитализации'].isna() 
                                 & ~df_obl['Дата фактической госпитализации из  ЕМИАС'].isna()
                                 & (df_obl['Дата фактической госпитализации из  ЕМИАС'] > pd.to_datetime("today"))
                             )

In [291]:
# -------------- фактическая дата первички _ГОСПИТ из будущего
df_obl.loc[   uslovie_наши_госпит 
         &    
          var_факт_из_будущ_госпит].shape

(0, 52)

In [292]:
# -------------- фактическая дата первички _ГОСПИТ из будущего
df_obl.loc[   uslovie_наши_госпит 
         &    
          var_факт_из_будущ_госпит, 'Правки' ] = '12 -фактическая дата первички _ГОСПИТ из будущего'

####
<div class="alert alert-block alert-info">    <font size=3>
<b>2. Проверка повторных визитов

In [293]:
# ------------------------------------------------------------------------------ НАШИ ППроверка повторных визитов -----------------------------
uslovie_2_наши_патронаж =    (     df_obl['Задание по пациенту'].isin(['отделение выездной паллиативной службы']) 
                                & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"')
                                & (df_obl['Текущий статус пациента'].isin(['На сопровождении', 'на сопровождении']) ) 
                             )

<div class="alert alert-block alert-info">    <font size=2>
13 - пустая дата последнего фактического визита у пациента на сопровождении ОВПП

In [294]:
# ---------------------------- не указан последний визит
var_нет_последн_визит =   df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                                

In [295]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_нет_последн_визит].shape

(866, 52)

In [296]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_нет_последн_визит, 'Правки' ] = '13 -пустая дата последнего фактического визита у пациента на сопровождении ОВПП'

<div class="alert alert-block alert-info">    <font size=2>
    14 дата последнего фактического визита у пациента на сопровождении ОВПП больше 30 дней назад без указания причины

In [297]:
# ---------------------------- последний визит более 30 без причина
var_более_30_нет_причин =  (  ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                            & ((pd.to_datetime("today") - df_obl['Последний визит врача из протокола осмотра ЕМИАС']) > '30 days') 
                            & (df_obl['Причина прекращения на сопровождении для пациентов на учете'].isna())
                           )     

In [298]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_более_30_нет_причин].shape

(1958, 52)

In [299]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_более_30_нет_причин, 'Правки' ] = '14 -дата последнего фактического визита у пациента на сопровождении ОВПП больше 30 дней назад без указания причины'

<div class="alert alert-block alert-info">    <font size=2>
    15 последний фактический визит из будущего

In [300]:
# ---------------------------- последний фактический визит из будущего
var_послед_из_будущ =  (  ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                            & (pd.to_datetime("today") < df_obl['Последний визит врача из протокола осмотра ЕМИАС'])  
                        )     

In [301]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_послед_из_будущ].shape

(0, 52)

In [302]:
# ---------------------------- не указан последний визит
df_obl.loc[   uslovie_2_наши_патронаж 
         &    
          var_послед_из_будущ, 'Правки' ] = '15 -последний фактический визит из будущего'

<div class="alert alert-block alert-warning">    <font size=1>

<div class="alert alert-block alert-info">    <font size=2>
16- последний фактический визит раньше первичного 24 ч

In [303]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
var_факт_ранее_первич_24 =  (    ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                              & ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
                              & (df_obl['Последний визит врача из протокола осмотра ЕМИАС'] < df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'])
                            )     

In [304]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
df_obl.loc[   uslovie_1_наши
         &    
          var_факт_ранее_первич_24].shape

(0, 52)

In [305]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
df_obl.loc[   uslovie_1_наши
         &    
          var_факт_ранее_первич_24, 'Правки' ] = '16 -последний фактический визит раньше первичного 24 ч'

<div class="alert alert-block alert-info">    <font size=2>
17- последний фактический визит раньше первичного 72 ч

In [306]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
var_факт_ранее_первич_72 =  (    ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                              & ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
                              & (df_obl['Последний визит врача из протокола осмотра ЕМИАС'] < df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'])
                            )     

In [307]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
df_obl.loc[   uslovie_1_наши
         &    
          var_факт_ранее_первич_72].shape

(0, 52)

In [308]:
# ---------------------------- 16- последний фактический визит раньше первичного 24 ч
df_obl.loc[   uslovie_1_наши
         &    
          var_факт_ранее_первич_72, 'Правки' ] = '17 -последний фактический визит раньше первичного 72 ч'

####
<div class="alert alert-block alert-info">    <font size=3>
<b>3. Проверка плановых визитов

In [309]:
uslovie_3 = (  
              (   (    df_obl['Задание по пациенту'].isin(['отделение выездной паллиативной службы']) 
                    & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"')
                   )
                 | (   ~df_obl['Задание по пациенту'].isin([ 'отделение выездной паллиативной службы', 'ОВПП (Ресурсный центр)']) 
                      & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"') 
                    )                                  
             )  
         &   ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
            )

In [310]:
uslovie_3_без_ресурс = (  
                           (    ~df_obl['Задание по пациенту'].isin(['ОВПП (Ресурсный центр)']) 
                                & (df_obl['Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях'] != 'БМЧУ "Детский хоспис"')
                            )
                         &   ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                      )

In [311]:
df_obl.loc[uslovie_3]                            .shape
# df_obl[uslovie_3_без_ресурс]                            .shape

(11334, 52)

<div class="alert alert-block alert-info">    <font size=2>
 18   пустая дата планового визита у пациента на сопровождении без указания причины

In [312]:
# ---------------------------- пустая дата планового визита у пациента на сопровождении без указания причины
var_дата_план_пусто_без_причин =  (  df_obl['Текущий статус пациента'].isin(['На сопровождении'])
                                    & df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'].isna()
                                    & df_obl['Причина прекращения на сопровождении для пациентов на учете'].isna()
                                   )     

In [313]:
# ---------------------------- пустая дата планового визита у пациента на сопровождении без указания причины
df_obl.loc[   uslovie_3 
         &    
          var_дата_план_пусто_без_причин].shape

(1193, 52)

In [314]:
# ---------------------------- пустая дата планового визита у пациента на сопровождении без указания причины
df_obl.loc[   uslovie_3_без_ресурс 
         &    
          var_дата_план_пусто_без_причин, 'Правки' ] = '18 -пустая дата планового визита у пациента на сопровождении без указания причины'

<div class="alert alert-block alert-info">    <font size=2>
<b>    19 дата планового визита раньше последнего фактического визита

In [315]:
# ---------------------------- дата планового визита раньше последнего фактического визита
var_дата_план_раньше_факт =  (   ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                               & ~df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'].isna()
                               & (df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'] < df_obl['Последний визит врача из протокола осмотра ЕМИАС'])
                             )

In [316]:
# ---------------------------- дата планового визита раньше последнего фактического визита
df_obl.loc[   uslovie_3_без_ресурс 
         &    
          var_дата_план_раньше_факт].shape

(0, 52)

In [317]:
# ---------------------------- дата планового визита раньше последнего фактического визита
df_obl.loc[   uslovie_3_без_ресурс 
         &    
          var_дата_план_раньше_факт, 'Правки' ] = '19 -дата планового визита раньше последнего фактического визита'

<div class="alert alert-block alert-info">    <font size=2>
    <b>20  плановая дата больше чем через месяц от последнего фактического визита без указания причины

In [318]:
# ---------------------------- плановая дата больше чем через месяц от последнего фактического визита без указания причины
var_план_более30_от_посл_факта_бе_причины =  (       ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna() 
                                                   & ~df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'].isna()
                                                   & ((df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'] - df_obl['Последний визит врача из протокола осмотра ЕМИАС']) > '30 days')
                                                   & df_obl['Текущий статус пациента'].isin(['На сопровождении'])
                                                   & df_obl['Причина прекращения на сопровождении для пациентов на учете'].isna()
                                                 )

In [319]:
# ---------------------------- плановая дата больше чем через месяц от последнего фактического визита без указания причины
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_план_более30_от_посл_факта_бе_причины].shape

(1523, 52)

In [320]:
# ---------------------------- плановая дата больше чем через месяц от последнего фактического визита без указания причины
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_план_более30_от_посл_факта_бе_причины, 'Правки' ] = '20 -плановая дата больше чем через месяц от последнего фактического визита без указания причины'

<div class="alert alert-block alert-info">    <font size=2>
    <b>21  приостановка с дефектом по первичке, но причина приостановки по первичке не указана  (какой ранее ???)

In [321]:
# ---------------- см.ранее

<div class="alert alert-block alert-info">    <font size=2>
<b>22 приостановка без дефекта по первичке, но причина приостановки на учете не указана

In [322]:
# ---------------------------- приостановка без дефекта по первичке, но причина приостановки на учете не указана
var_без_деф_первич_но_прич_приостанов_не_указана =  (      df_obl['Текущий статус пациента'].isin(['Сопровождение приостановлено','сопровождение приостановлено',])
                                                  & df_obl['Причина прекращения на сопровождении для пациентов на учете'].isna()
                                                  & df_obl['Причины невыполнения «задания по пациенту» для первичных пациентов'].isna()
                                                 )

In [323]:
# ---------------------------- приостановка без дефекта по первичке, но причина приостановки на учете не указана
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_без_деф_первич_но_прич_приостанов_не_указана].shape

(1, 52)

In [324]:
# ---------------------------- приостановка без дефекта по первичке, но причина приостановки на учете не указана
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_без_деф_первич_но_прич_приостанов_не_указана, 'Правки' ] = '22 -приостановка без дефекта по первичке, но причина приостановки на учете не указана'

<div class="alert alert-block alert-info">    <font size=2>
<b>23 дата прекращения из будущего

In [325]:
# ---------------------------- дата прекращения из будущего
var_дата_прекращ_из_будущ =  (   df_obl['Дата прекращения оказания ПМП'] > (pd.to_datetime("today") ))
                                      

In [326]:
# ---------------------------- дата прекращения из будущего
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_дата_прекращ_из_будущ].shape

(0, 52)

In [327]:
# ---------------------------- дата прекращения из будущего
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_дата_прекращ_из_будущ, 'Правки' ] = '23 -дата прекращения из будущего'

<div class="alert alert-block alert-info">    <font size=2>
<b>24 оказание ПМП прекращено и нет даты и/или причины прекращения

In [328]:
# ---------------------------- оказание ПМП прекращено и нет даты и/или причины прекращения
var_стоп_ПМП_но_нет_дат_причи_прекращ =  (   df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                            & 
                                              (  df_obl['Причина прекращения оказания ПМП'].isna()  | df_obl['Дата прекращения оказания ПМП'].isna())
                                         )

In [329]:
# ---------------------------- оказание ПМП прекращено и нет даты и/или причины прекращения
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_нет_дат_причи_прекращ].shape

(0, 52)

In [330]:
# ---------------------------- оказание ПМП прекращено и нет даты и/или причины прекращения
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_нет_дат_причи_прекращ, 'Правки' ] = '24 -оказание ПМП прекращено и нет даты и/или причины прекращения'

<div class="alert alert-block alert-info">    <font size=2>
<b>25 оказание ПМП прекращено, но есть плановый визит

In [331]:
# ---------------------------- оказание ПМП прекращено, но есть плановый визит
var_стоп_ПМП_но_есть_план_визит =  (   df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                     & ~df_obl['Плановая дата следующего визита из протокола осмотра ЕМИАС'].isna() 
                                   )

In [332]:
# ---------------------------- оказание ПМП прекращено, но есть плановый визит
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_есть_план_визит].shape

(1648, 52)

In [333]:
# ---------------------------- оказание ПМП прекращено, но есть плановый визит
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_есть_план_визит, 'Правки' ] = '25 -оказание ПМП прекращено, но есть плановый визит'

<div class="alert alert-block alert-info">    <font size=2>
<b>26 оказание ПМП прекращено, но заполнена причина приостановки на учете

In [334]:
# ---------------------------- оказание ПМП прекращено, но заполнена причина приостановки на учете
var_стоп_ПМП_но_есть_прич_приостанов =  (   df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                         & ~df_obl['Причина прекращения на сопровождении для пациентов на учете'].isna() 
                                   )

In [335]:
# ---------------------------- оказание ПМП прекращено, но заполнена причина приостановки на учете
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_есть_прич_приостанов].shape

(0, 52)

In [336]:
# ---------------------------- оказание ПМП прекращено, но заполнена причина приостановки на учете
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стоп_ПМП_но_есть_прич_приостанов, 'Правки' ] = '26 -оказание ПМП прекращено, но заполнена причина приостановки на учете'

<div class="alert alert-block alert-info">    <font size=2>
<b>27 статус НЕ прекращено оказание ПМП, но заполнена причина прекращения и/или дата прекращения и/или дата смерти

In [337]:
# ---------------------------- статус НЕ прекращено оказание ПМП, но заполнена причина прекращения и/или дата прекращения и/или дата смерти
var_НЕстопПМП_но_есть_прич_прекращ_или_дата_прекр_или_смерти =  (   ~df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                                                     & 
                                                                       ( ~df_obl['Причина прекращения оказания ПМП'].isna() 
                                                                        |~df_obl['Дата прекращения оказания ПМП'].isna() 
                                                                        |~df_obl['Дата смерти'].isna() 
                                                                       ) 
                                                                  )

In [338]:
# ---------------------------- статус НЕ прекращено оказание ПМП, но заполнена причина прекращения и/или дата прекращения и/или дата смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_НЕстопПМП_но_есть_прич_прекращ_или_дата_прекр_или_смерти].shape

(1, 52)

In [339]:
# ---------------------------- статус НЕ прекращено оказание ПМП, но заполнена причина прекращения и/или дата прекращения и/или дата смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_НЕстопПМП_но_есть_прич_прекращ_или_дата_прекр_или_смерти, 'Правки' ] = '27 -статус НЕ прекращено оказание ПМП, но заполнена причина прекращения и/или дата прекращения и/или дата смерти'

<div class="alert alert-block alert-info">    <font size=2>
<b>28 дата смерти из будущего

In [340]:
# ----------------------------  дата смерти из будущего
var_дата_смерти_из_будущего =  (         ~df_obl['Дата смерти'].isna() 
                                                                   &   (  df_obl['Дата смерти'] > pd.to_datetime("today") )
                                                                        
                                                                  )

In [341]:
# ----------------------------  дата смерти из будущего
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_дата_смерти_из_будущего].shape

(0, 52)

In [342]:
# ----------------------------  дата смерти из будущего
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_дата_смерти_из_будущего, 'Правки' ] = '28- дата смерти из будущего'

<div class="alert alert-block alert-info">    <font size=2>
<b>29 оказание ПМП прекращено, причина - смерть или смерть до первичного визита, но нет даты смерти

In [343]:
# ---------------------------- оказание ПМП прекращено, причина - смерть или смерть до первичного визита, но нет даты смерти
var_стопПМП_прич_смерть_или_до_по_но_нет_даты_смерти =  (    df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                                          & df_obl['Причина прекращения оказания ПМП'].isin(['смерть пациента',
                                                                                                                   'Смерть пациента до даты первичного визита врача (госпитализации)', 
                                                                                                                   'Смерть пациента', 
                                                                                                                   'смерть пациента до даты первичного визита врача (госпитализации)']) 
                                                           & df_obl['Дата смерти'].isna() 
                                                                
                                                        )

In [344]:
# ----------------------------  оказание ПМП прекращено, причина - смерть или смерть до первичного визита, но нет даты смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_или_до_по_но_нет_даты_смерти].shape

(0, 52)

In [345]:
# ----------------------------  оказание ПМП прекращено, причина - смерть или смерть до первичного визита, но нет даты смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_или_до_по_но_нет_даты_смерти, 'Правки' ] = '29 -оказание ПМП прекращено, причина - смерть или смерть до первичного визита, но нет даты смерти'

<div class="alert alert-block alert-info">    <font size=2>
<b>30 оказание ПМП прекращено, причина - смерть, но нет первичного визита

In [346]:
# ---------------------------- оказание ПМП прекращено, причина - смерть, но нет первичного визита
var_стопПМП_прич_смерть_но_нет_первички =  (    df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                              & df_obl['Причина прекращения оказания ПМП'].isin(['смерть пациента',
                                                                                                 # 'Смерть пациента до даты первичного визита врача (госпитализации)', 'смерть пациента до даты первичного визита врача (госпитализации)'
                                                                                                 'Смерть пациента', 
                                                                                                   ]) 
                                               & (df_obl['Дата фактической госпитализации из  ЕМИАС'].isna() & df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna())
                                                        )

In [347]:
# ----------------------------  оказание ПМП прекращено, причина - смерть, но нет первичного визита
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_но_нет_первички].shape

(0, 52)

In [348]:
# ----------------------------  оказание ПМП прекращено, причина - смерть, но нет первичного визита
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_но_нет_первички, 'Правки' ] = '30 -оказание ПМП прекращено, причина - смерть, но нет первичного визита'

<div class="alert alert-block alert-info">    <font size=2>
<b>31 оказание ПМП прекращено, причина - смерть до первичного визита, но есть первичного визита или последний визит

In [349]:
# ---------------------------- оказание ПМП прекращено, причина - смерть до первичного визита, но есть первичного визита или последний визит
var_стопПМП_прич_смерть_доПО_но_есть_первичка_или_вторич =  (    df_obl['Текущий статус пациента'].isin(['Оказание ПМП прекращено', 'оказание ПМП прекращено',])
                                                               & df_obl['Причина прекращения оказания ПМП'].isin([
                                                                                                             # 'смерть пациента', 'Смерть пациента', 
                                                                                                             'Смерть пациента до даты первичного визита врача (госпитализации)', 'смерть пациента до даты первичного визита врача (госпитализации)'
                                                                                                                 ]) 
                                                               &  ( ~df_obl['Дата фактической госпитализации из  ЕМИАС'].isna() 
                                                                  | ~df_obl['Последний визит врача из протокола осмотра ЕМИАС'].isna()
                                                                  | ~df_obl['Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС'].isna()
                                                                  | ~df_obl['Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС'].isna()
                                                             
                                                                    )
                                                            )

In [350]:
# ----------------------------  оказание ПМП прекращено, причина - смерть до первичного визита, но есть первичного визита или последний визит
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_доПО_но_есть_первичка_или_вторич].shape

(0, 52)

In [351]:
# ----------------------------  оказание ПМП прекращено, причина - смерть до первичного визита, но есть первичного визита или последний визит
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_стопПМП_прич_смерть_доПО_но_есть_первичка_или_вторич, 'Правки' ] = '31 -оказание ПМП прекращено, причина - смерть до первичного визита, но есть первичного визита или последний визит'

<div class="alert alert-block alert-info">    <font size=2>
<b>32 проверка наличия визита после даты смерти

In [352]:
# ---------------------------- проверка наличия визита после даты смерти
var_наличие_визита_после_смерти =  (    ~df_obl['Дата смерти'].isna()
                                                               &  (  df_obl['Дата смерти'] < df_obl['Последний визит врача из протокола осмотра ЕМИАС']))

In [353]:
# df_obl.columns.to_list()

In [354]:
# ----------------------------  проверка наличия визита после даты смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_наличие_визита_после_смерти].shape

(17, 52)

In [355]:
# df_obl.loc[ 
#     uslovie_3_без_ресурс 
#          &    
#           var_наличие_визита_после_смерти]

In [356]:
# ----------------------------  проверка наличия визита после даты смерти
df_obl.loc[ 
    uslovie_3_без_ресурс 
         &    
          var_наличие_визита_после_смерти, 'Правки' ] = '32 -есть визит после даты смерти'

In [357]:
df_obl.columns.to_list()

['Дата включения пациента в Реестр',
 'ФИО пациента',
 'ID пациента',
 'Пол',
 'Дата рождения',
 'Гражданский статус',
 'Адрес регистрации по месту жительства',
 'Адрес регистрации по месту пребывания в Москве или адрес фактического места жительства в Москве',
 'Вид документа',
 'Номер документа, удостоверяющего личность',
 'СНИЛС',
 'Полис ОМС',
 'Инвалидность',
 'ФИО контактного лица',
 'Степень родства контактного лица',
 'Контактный телефон контактного лица',
 'Наличие социального работника',
 'Контактные данные социального работника',
 'Прикрепление к патронажному участку ',
 'Дата прикрепления к патронажному участку',
 'МО патронажного прикрепления',
 'МО прикрепления онкологического профиля',
 'Медицинское заключение о наличии у пациента показаний для оказания паллиативной медицинской помощи ',
 'Дата выдачи заключения',
 'Наименование медицинской организации, выдавшей медицинское заключение',
 'Разрешение ДЗМ на оказание паллиативной медицинской помощи пациенту ',
 'Дата выдачи

<div class="alert alert-block alert-warning">    <font size=2>
<b>вывод в БД на дашборд

In [358]:
from sqlalchemy import create_engine

# Подключаемся к базе данных PGSQL
file_path = 'Z:/Отдел аналитики/05_Шаблоны_пандас/Александр/CPP_date/engine_dit.txt' # ДИТ
with open(file_path) as file:
    engine_path = file.read()
engine = create_engine(engine_path)

df_obl = df_obl.rename(columns={
                    'Фактическая дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС': 'Фактическая 24 дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС',
                    'Фактическая дата первичного выезда врача 72 часа из протокола осмотра ЕМИАС': 'Фактическая 72 дата первичного выезда врача 24 часа из протокола осмотра ЕМИАС',
                    'Фактическая дата первичного выезда врача 24 часа' : 'Фактическая 24 дт первичного выезда врача 24 часа',
                    'Фактическая дата первичного выезда врача 72 часа' : 'Фактическая 72 дт первичного выезда врача 72 часа',
                    })

df_obl.to_sql('obl_defect', con=engine, schema = 'public', if_exists='replace', index = False)

878

<div class="alert alert-block alert-success">    <font size=2>
<b>Тотал

In [359]:
pd.pivot_table(df_obl,
               index=['Правки', ],
               columns=["Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях"],
               # values=["Quantity", "Price"],
               aggfunc={"ID пациента": len,
                                          },
               margins=False,
               fill_value=0)

ID пациента  \
Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях БМЧУ "Детский хоспис"   
Правки                                                                                                                                                                
01 -просрочка по первичке 24 ч и нет причины и/...                                                                                                                0   
03 -фактическая дата первички 24 ч меньше плано...                                                                                                                0   
05 -просрочка по первичке 72 ч и нет причины и/...                                                                                                                0   
06 -ЗАВТРА просрочка по первичке 72 ч и нет при...                                                                                                                0   
07 -фактическая дата первички 24 ч меньше плано...                                                                                                                0   
10 -ЗАВТРА просрочка по первичке госп и нет при...                                                                                                                1   
11 -фактическая дата первичной госп меньше план...                                                                                                                0   
13 -пустая дата последнего фактического визита ...                                                                                                                0   
14 -дата последнего фактического визита у пацие...                                                                                                                0   
18 -пустая дата планового визита у пациента на ...                                                                                                                0   
20 -плановая дата больше чем через месяц от пос...                                                                                                                0   
25 -оказание ПМП прекращено, но есть плановый в...                                                                                                                0   
27 -статус НЕ прекращено оказание ПМП, но запол...                                                                                                                0   
32 -есть визит после даты смерти                                                                                                                                  0   

                                                                                                                                                           \
Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях Бутово ОВПП   
Правки                                                                                                                                                      
01 -просрочка по первичке 24 ч и нет причины и/...                                                                                                      0   
03 -фактическая дата первички 24 ч меньше плано...                                                                                                      0   
05 -просрочка по первичке 72 ч и нет причины и/...                                                                                                     38   
06 -ЗАВТРА просрочка по первичке 72 ч и нет при...                                                                                                      2   
07 -фактическая дата первички 24 ч меньше плано...                                                                                                     52   
10 -ЗАВТРА просрочка по первичке госп и нет при...      